# Notebook for Splunk Machine Learning Toolkit Container for TensorFlow

This notebook contains an example workflow for how to forecast a univariate time series. 

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("TensorFlow version: " + tf.__version__)
print("Keras version: " + keras.__version__)

numpy version: 1.18.1
pandas version: 1.0.1
TensorFlow version: 2.1.0
Keras version: 2.2.4-tf


## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a prepared dataset into this environment.

| inputlookup internet_traffic.csv 
| timechart span=120min avg("bits_transferred") as bits_transferred 
| eval bits_transferred=round(bits_transferred) 
| fit RobustScaler bits_transferred 
| fit MLTKContainer algo=forecast_rnn_internet_traffic mode=stage epochs=100 batch_size=3 RS_bits_transferred _time by RS_bits_transferred into app:internet_traffic_forecast_model

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("internet_traffic_forecast_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [3]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [4]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
df, param = stage("internet_traffic_forecast_model")
print(df[0:1])
print(df.shape)
print(str(param))
df[param['options']['split_by']]

        _time  RS_bits_transferred
0  1118152800             0.714191
(616, 2)
{'options': {'params': {'mode': 'stage', 'algo': 'univariate_cnn_forecast', 'epochs': '10', 'batch_size': '100', 'future_steps': '30', 'holdback': '30'}, 'args': ['RS_bits_transferred', '_time'], 'feature_variables': ['RS_bits_transferred', '_time'], 'split_by': ['RS_bits_transferred'], 'model_name': 'internet_traffic_forecast_model', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'disabled': False, 'handle_new_cat': 'default', 'max_distinct_cat_values': '1000', 'max_distinct_cat_values_for_classifiers': '1000', 'max_distinct_cat_values_for_scoring': '1000', 'max_fit_time': '6000', 'max_inputs': '100000000', 'max_memory_usage_mb': '4000', 'max_model_size_mb': '150', 'max_score_time': '6000', 'streaming_apply': '0', 'use_sampling': '1'}, 'kfold_cv': None}, 'feature_variables': ['RS_bits_transferred', '_time']}


,RS_bits_transferred
0,0.714191
1,1.162206
2,1.268620
3,1.268984
4,1.038817
...,...
611,-0.388140
612,-0.347570
613,0.120611
614,0.864492


## Stage 2 - create and initialize a model

In [5]:
# initialize the model
# params: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    # Collect variables
    model_batch_size = 3
    n_features = 1
    hidden_layers = 50
    activation_func = 'sigmoid'
    if 'options' in param:
        if 'params' in param['options']:
            if 'batch_size' in param['options']['params']:
                model_batch_size = int(param['options']['params']['batch_size'])
            if 'hidden_layers' in param['options']['params']:
                hidden_layers = int(param['options']['params']['hidden_layers'])
            if 'activation' in param['options']['params']:
                activation_func = param['options']['params']['activation']
    
    # define model
    model = keras.Sequential()
    model.add(keras.layers.LSTM(hidden_layers, activation=activation_func, input_shape=(model_batch_size, n_features)))
    model.add(keras.layers.Dense(n_features))
    model.compile(optimizer='adam', loss='mse')
    return model

In [6]:
model = init(df,param)
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50)                10400     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________
None


## Stage 3 - fit the model

In [7]:
# returns a fit info json object
# split a univariate sequence into samples
def split_sequence(sequence, batch_size):
    
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + batch_size
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

def fit(model,df,param):
    returns = {}
    
    # Collect variables from param file
    model_epochs = 100
    model_batch_size = 3
    holdback = 30
    if 'options' in param:
        if 'params' in param['options']:
            if 'epochs' in param['options']['params']:
                model_epochs = int(param['options']['params']['epochs'])
            if 'batch_size' in param['options']['params']:
                model_batch_size = int(param['options']['params']['batch_size'])
            if 'holdback' in param['options']['params']:
                holdback = int(param['options']['params']['holdback'])
    
    
    # flatten data frame into an array and extract the training set
    full_data = df[param['options']['split_by']].values.tolist()
    train_set = list(full_data[:len(full_data)-holdback])
    
    # split data into samples
    X, y = split_sequence(train_set, model_batch_size)
    # reshape from [samples, timesteps] into [samples, timesteps, features]
    n_features = 1
    X = X.reshape((X.shape[0], X.shape[1], 1))
    

    # connect model training to tensorboard
    log_dir="/srv/notebooks/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    
    # run the training
    returns['fit_history'] = model.fit(x=X,
                                       y=y, 
                                       verbose=2, 
                                       epochs=model_epochs,
                                       shuffle=False)
    # memorize parameters
    returns['model_epochs'] = model_epochs
    returns['model_batch_size'] = model_batch_size
    returns['model_loss_acc'] = model.evaluate(x = X, y = y)
    return returns

In [8]:
returns = fit(model,df,param)
print(returns['model_loss_acc'])

Train on 486 samples
Epoch 1/10
486/486 - 2s - loss: 0.3680
Epoch 2/10
486/486 - 1s - loss: 0.3463
Epoch 3/10
486/486 - 2s - loss: 0.3306
Epoch 4/10
486/486 - 2s - loss: 0.3162
Epoch 5/10
486/486 - 1s - loss: 0.3023
Epoch 6/10
486/486 - 1s - loss: 0.2888
Epoch 7/10
486/486 - 1s - loss: 0.2756
Epoch 8/10
486/486 - 1s - loss: 0.2628
Epoch 9/10
486/486 - 1s - loss: 0.2502
Epoch 10/10
486/486 - 1s - loss: 0.2379
486/486 [==============================] - 1s 2ms/sample - loss: 0.2304
0.230404122130861


## Stage 4 - apply the model

In [11]:
def apply(model,df,param):
    
    # Collect variables
    model_batch_size = 3
    future_steps = 30
    holdback = 30
    if 'options' in param:
        if 'params' in param['options']:
            if 'batch_size' in param['options']['params']:
                model_batch_size = int(param['options']['params']['batch_size'])
            if 'future_steps' in param['options']['params']:
                future_steps = int(param['options']['params']['future_steps'])
            if 'holdback' in param['options']['params']:
                holdback = int(param['options']['params']['holdback'])
    
    # select training data
    X = df[param['options']['split_by']].values

    test_set = X[len(X)-holdback-model_batch_size:]
    predictions = list(X[:len(X)-holdback])
    # generate forecast
    for i in range(0, holdback+future_steps):
        if i<holdback:
            X_batch = test_set[i:i+model_batch_size].reshape(1,model_batch_size,1)
            y_pred = model.predict(x = X_batch, verbose=1)
            predictions.append(list(y_pred[0]))
        else:
            X_batch = test_set[i:i+model_batch_size].reshape(1,model_batch_size,1)
            y_pred = model.predict(x = X_batch, verbose=1)
            predictions.append(list(y_pred[0]))
            test_set = np.append(test_set, y_pred[0])
            
    # append predictions to time series to return a data frame
    return predictions

In [12]:
training_instance = apply(model,df,param)
print(len(training_instance))
training_instance[583:]

1/1 [==============================] - 0s 33ms/sample
646


[array([0.328399]),
 array([0.18502462]),
 array([-9.24839172e-05]),
 [0.19981584],
 [0.1255655],
 [0.08042667],
 [0.06335317],
 [0.11178725],
 [0.3096624],
 [0.43013453],
 [0.44793108],
 [0.4651054],
 [0.38016388],
 [0.2524955],
 [0.2139458],
 [0.16274178],
 [0.0985539],
 [0.06007012],
 [0.051599935],
 [0.14013197],
 [0.33504146],
 [0.40459907],
 [0.44544217],
 [0.43764862],
 [0.3405208],
 [0.23911276],
 [0.1955782],
 [0.14060003],
 [0.0832598],
 [0.05837349],
 [0.056385558],
 [0.16167833],
 [0.35194933],
 [0.4293462],
 [0.3006632],
 [0.26334992],
 [0.25059432],
 [0.24428393],
 [0.24021266],
 [0.23722959],
 [0.23493493],
 [0.23313665],
 [0.2317152],
 [0.23058543],
 [0.22968386],
 [0.22896214],
 [0.22838259],
 [0.22791605],
 [0.22753976],
 [0.22723563],
 [0.22698924],
 [0.22678956],
 [0.22662733],
 [0.22649543],
 [0.22638811],
 [0.2263006],
 [0.2262291],
 [0.22617073],
 [0.22612315],
 [0.22608417],
 [0.22605227],
 [0.22602619],
 [0.22600485]]

## Stage 5 - save the model

In [ ]:
# save model to name in expected convention "<algo_name>_<model_name>.h5"
def save(model,name):
    # save keras model to hdf5 file
    # https://www.tensorflow.org/beta/tutorials/keras/save_and_restore_models
    model.save(MODEL_DIRECTORY + name + ".h5")
    return model

## Stage 6 - load the model

In [ ]:
# load model from name in expected convention "<algo_name>_<model_name>.h5"
def load(name):
    model = keras.models.load_model(MODEL_DIRECTORY + name + ".h5")
    return model

## Stage 7 - provide a summary of the model

In [ ]:
# return model summary
def summary(model=None):
    returns = {"version": {"tensorflow": tf.__version__, "keras": keras.__version__} }
    if model is not None:
        # Save keras model summary to string:
        s = []
        model.summary(print_fn=lambda x: s.append(x+'\n'))
        returns["summary"] = ''.join(s)
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code